<a href="https://colab.research.google.com/github/daniel-falk/ai-ml-principles-exercises/blob/main/ML-training/intro-to-libraries/intro_to_tensorflow_low_level.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using TensorFlow with the low-level API
For easy use of TensorFlow to build and train neural network models, see the intro to Keras example. This example will instead show the low-level API in TensorFlow which can be used to perform advanced operations such as gradient calculation and manual backpropagation.

Note that the API changed significantly between TF 1 and TF2, so make sure that you are using version 2.x

In [ ]:
import tensorflow as tf

tf.__version__

In [ ]:
import numpy as np

# Start with creating a simpel dataset, the features could be
# e.g. the different genres a movie can be given.
movie_genres = np.array([
    [0, 1, 1],
    [1, 1, 1],
    [1, 1, 0],
    [0, 1, 0],
    [1, 0, 1]],
    dtype='float32')

# While the ground-truths are the ratings they have got by the viewers
movie_ratings = np.array([
    [1.0],
    [0.5],
    [0.0],
    [0.0],
    [0.7]],
    dtype='float32')

In [ ]:
# The prediction is based on weights, we start by setting them randomly
w = tf.Variable(tf.random.normal([3, 1], stddev=0.3), name='weights')
w0 = tf.identity(w) # copy the tensor values for later use
w0

In [ ]:
# We then calculate the predicted ratings as w*movie_genres and measures
# with our loss function how far that is from the truth
with tf.GradientTape() as tape:
  prediction = tf.sigmoid(tf.matmul(movie_genres, w), name='output')
  loss = tf.losses.mean_squared_error(movie_ratings, prediction)

In [ ]:
# Since using the gradient tape, the derivative of the loss
# with respect to the weights was automatically calculated
gradients = tape.gradient(loss, w)

In [ ]:
p0 = tf.identity(prediction)
l0 = tf.identity(loss)
g0 = tf.identity(gradients)
print("Prediction:", p0)
print("Loss:", l0)
print("Gradients:", g0)

In [ ]:
# To take one step closer towards the ground-truth ratings, we can
# update the weights as w = w - lr * gradients
learning_rate = tf.constant(0.01)
update_weights = w.assign(w - tf.multiply(learning_rate, gradients), name='update_weights')

In [ ]:
# Use the updated weights to make a new prediction of the ratings
prediction = tf.sigmoid(tf.matmul(movie_genres, w), name='output')
loss = tf.losses.mean_squared_error(movie_ratings, prediction)

In [ ]:
print(f"Prediction: {tf.reshape(prediction, -1)}")
print(f"Change in prediction: {tf.reshape(prediction - p0, -1)}")
print(f"Loss: {loss}")
print(f"Change of loss: {loss - l0}")

We can see that the loss for all predicted sampels was smaller after updating the weights. If we do this multiple times we will get closer and closer to the ground truth ratings. This iterative update is called a training loop.